# Exam project of group 26
Exam numbers:
- 144
- 161
- 177
- 193

### All of the dependency imports

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import re

import time
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains
from collections import defaultdict
import scraping_class
from statsmodels.tsa.tsatools import lagmat

from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, validation_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sktime.forecasting.model_selection import ExpandingWindowSplitter, ForecastingGridSearchCV
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.base import ForecastingHorizon

from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score, median_absolute_error
from sklearn.metrics import make_scorer

The below makes a header for all HTTP requests using the scraping_class. If you already have a file named log.csv in your folder, then the below might fail due to a bug in scraping_class

In [ ]:
s = requests.Session()
# update headers to the requests
s.headers.update({'contact_details': 'qxd466@alumni.ku.dk', 'purpose': "Dear recipient, we've scraped your data with the purpose of creating an academic report on electricity prices and if it's possible to predict these. Please contact us via the contact details, if there's any issue with this. Best regards."})
conn = scraping_class.Connector(logfile="log.csv", overwrite_log=False, session = s)

# Data Collection and Wrangling
The below collection and wrangling is all handled by functions to make the this process more seamless and reduce the number of globally defined variables to reduce the burden on RAM

# Nordpool data


We identify the [webpage](https://www.nordpoolgroup.com/historical-market-data/) with the desired information stored as seperate HTML files.

However, we are not able to scrape Nordpool to obtain the individual links for the files as the webpage call an internal API. Luckily we are able to call the API and obtain the JSON. 

## Below we define function for data gathering and handling of all Nordpool data

In [ ]:
### We define a function for getting the electricity spot prices ###
def getElspotPrices():
    """ This function downloads the Elspot prices from Nordpool, deletes unnecessary elements from the html response
    and then adds datetime as index"""
    # Get the json data from Nordpool
    url = 'https://www.nordpoolgroup.com/api/downloads/4675'
    el_res, _ = conn.get(url, "Nordpool Data")
    el_json = el_res.json()

    # Selects the urls for the hourly elspot prices in DKK
    hourly = [i for i in el_json['Files'] if i['Resolution'] == 'Hourly']
    elspot = [i for i in hourly if i['Categories'] == ['Elspot Prices']]
    elspot = [i for i in elspot if i['Name'].endswith('DKK')]
    elspot_url = ['https://www.nordpoolgroup.com'+i['Url'] for i in elspot]

    # Create a dictonary with all of the data for each year and put it into a DataFrame
    d = {}
    for i in range(len(elspot_url)):
        d[f'DF_{2013+i}'] = pd.read_html(elspot_url[i])[0].droplevel(level = 0, axis = 1).droplevel(level = 0, axis = 1)

    # Sometimes Nordpool makes changes to their 2021 dataset, and this leads to them adding another header level.
    # The following removes this header level, if this is present
    try:
        # Standardize the 2021 dataset
        d['DF_2021'] = d['DF_2021'].droplevel(level = 0, axis = 1).rename(columns= {'Unnamed: 0_level_3': 'Unnamed: 0_level_2'})
    except ValueError:
        pass

    # Concat the DataFrames into one DF
    df_concat = pd.concat(d)

    # Structure the DataFrame
    df_new = df_concat[['Unnamed: 0_level_2', 'Hours','DK2']].droplevel(level = 0).reset_index(drop = True)
    df_new[df_new.columns[2:]] = df_new[df_new.columns[2:]]/100
    df_new = df_new.rename(columns = {'Unnamed: 0_level_2': 'date'})

    # Add datetime columns
    df_new['date'] = pd.to_datetime(df_new['date'], format = '%d-%m-%Y')
    df_new['hour'] = df_new['Hours'].str[5:]
    df_new.drop('Hours', axis = 1, inplace = True)
    df_new['hour'] = pd.to_datetime(df_new['hour'], format = '%H').dt.hour

    # Make the index into datetime
    df_new.index = pd.to_datetime(df_new['date'].apply(str)+' '+pd.to_datetime(df_new['hour'], format = '%H').dt.time.apply(str))

    return df_new

### We define a function for getting the electricity consumption prognosis ###
def ConsumptionPrognosis():
    """Function that gets all the available electricity consumption prognoses from nordpool
    and does some initial data wrangling, e.g., deletes unnecessary elements and adds a datetime index"""
    # Get the json from the internal API on Nordpool.com
    url = "https://www.nordpoolgroup.com/api/downloads/4675"
    res, _ = conn.get(url, "Nordpool Data")
    page = res.json()

    # Get the URLs for the html files containing the hourly consumption prognosis' and transform them into a list
    # of dataframes 
    hourly = [i for i in page["Files"] if i["Resolution"] == "Hourly"]
    Categories = [i for i in hourly if i["Categories"] == ["Consumption prognosis"]]
    DKK = [i for i in Categories if "DK" in i["Name"]]
    URLS_CP = ["https://www.nordpoolgroup.com"+i["Url"] for i in DKK]
    df_con_prog = [pd.read_html(i)[0].droplevel(level = 0, axis = 1).droplevel(level = 0, axis = 1) for i in URLS_CP]

    # Concatenate the dataframes, rename, and format time stamp
    df_CP = pd.concat(df_con_prog)
    cols = ["Unnamed: 0_level_2", "Hours", "DK1", "DK2"]
    df_CP = df_CP[cols].rename(columns = {"Unnamed: 0_level_2": "date", "DK1": "DK1_CP", "DK2": "DK2_CP"})
    df_CP["date"] = pd.to_datetime(df_CP["date"], format = "%d-%m-%Y")
    df_CP["hour"] = df_CP["Hours"].str[5:]
    df_CP.drop('Hours', axis = 1, inplace = True)
    df_CP["hour"] = pd.to_datetime(df_CP["hour"], format = '%H').dt.hour

    # Set index equal to time stamp 
    df_CP.index = pd.to_datetime(df_CP["date"].apply(str)+' '+pd.to_datetime(df_CP["hour"], format = '%H').dt.time.apply(str))

    return df_CP

### Production Prognosis ###
# We extract the production prognosis. The decimal pointer are correct for the consumption prognosis data sets.
def ProductionPrognosis(): 
    """Function that gets all the available electricity production prognoses from nordpool
    and does some initial data wrangling, e.g., deletes unnecessary elements and adds a datetime index"""
    # Get the json from the internal API on Nordpool.com
    url = "https://www.nordpoolgroup.com/api/downloads/4675"
    res, _ = conn.get(url, "Nordpool Data")
    page = res.json()

    # Get the URLs for the html files containing the hourly consumption prognosis' and transform them into a list
    # of dataframes 
    hourly = [i for i in page["Files"] if i["Resolution"] == "Hourly"]
    Categories = [i for i in hourly if i["Categories"] == ["Production prognosis"]]
    URLS_PP = ["https://www.nordpoolgroup.com"+i["Url"] for i in Categories]
    df_pro_prog = [pd.read_html(i)[0].droplevel(level = 0, axis = 1).droplevel(level = 0, axis = 1) for i in URLS_PP]

    # Concatenate the dataframes, rename, and format time stamp
    df_PP = pd.concat(df_pro_prog)
    cols = ["Unnamed: 0_level_2", "Hours", "DK1", "DK2"]
    df_PP = df_PP[cols].rename(columns = {"Unnamed: 0_level_2": "date", "DK1": "DK1_PP", "DK2": "DK2_PP"})
    df_PP["date"] = pd.to_datetime(df_PP["date"], format = "%d-%m-%Y")
    df_PP["hour"] = df_PP["Hours"].str[5:]
    df_PP.drop('Hours', axis = 1, inplace = True)
    df_PP["hour"] = pd.to_datetime(df_PP["hour"], format = '%H').dt.hour


    # Set index equal to time stamp 
    df_PP.index = pd.to_datetime(df_PP["date"].apply(str)+' '+pd.to_datetime(df_PP["hour"], format = '%H').dt.time.apply(str))

    return df_PP

### Wind Power Prognosis ###
# We extract the wind power prognosis. The decimal pointer are correct for the consumption prognosis data sets.
def WindPrognosis():
    """ Function that gets all the available wind prognoses from nordpool
    and does some initial data wrangling, e.g., deletes unnecessary elements and adds a datetime index"""
    # Get the json from the internal API on Nordpool.com
    url = "https://www.nordpoolgroup.com/api/downloads/4675"
    res, _ = conn.get(url, "Nordpool Data")
    page = res.json()

    # Get the URLs for the html files containing the hourly consumption prognosis' and transform them into a list
    # of dataframes 
    hourly = [i for i in page["Files"] if i["Resolution"] == "Hourly"]
    Categories = [i for i in hourly if i["Categories"] == ["Wind power prognosis"]]
    DKK = [i for i in Categories if "DK" in i["Name"]]
    URLS_WP = ["https://www.nordpoolgroup.com"+i["Url"] for i in DKK]
    df_wind_prog = [pd.read_html(i)[0].droplevel(level = 0, axis = 1).droplevel(level = 0, axis = 1) for i in URLS_WP]

    # Concatenate the dataframes, rename, and format time stamp
    df_WP = pd.concat(df_wind_prog)
    cols = ["Unnamed: 0_level_2", "Hours", "DK1", "DK2"]
    df_WP = df_WP[cols].rename(columns = {"Unnamed: 0_level_2": "date", "DK1": "DK1_WP", "DK2": "DK2_WP"})
    df_WP["date"] = pd.to_datetime(df_WP["date"], format = "%d-%m-%Y")
    df_WP["hour"] = df_WP["Hours"].str[5:]
    df_WP.drop('Hours', axis = 1, inplace = True)
    df_WP["hour"] = pd.to_datetime(df_WP["hour"], format = '%H').dt.hour


    # Set index equal to time stamp 
    df_WP.index = pd.to_datetime(df_WP["date"].apply(str)+' '+pd.to_datetime(df_WP["hour"], format = '%H').dt.time.apply(str))

    return df_WP

### Power Exchange ###
# We extract the power exchange. We change the value og observation to fix the misplaced decimal pointer when relevant. 
def ExchangeConnections():
    """ Function that gets all the available electricity exchange connections from nordpool
    and does some initial data wrangling, e.g., deletes unnecessary elements, adds decimal pointer and adds a datetime index """
    # Get the json from the internal API on Nordpool.com
    url = "https://www.nordpoolgroup.com/api/downloads/4675"
    res, _ = conn.get(url, "Nordpool Data")
    page = res.json()

    # Get the URLs for the html files containing the hourly consumption prognosis' and transform them into a list
    # of dataframes 
    hourly = [i for i in page["Files"] if i["Resolution"] == "Hourly"] 
    Categories = [i for i in hourly if i["Categories"] == ["Exchange connections"]]
    DKK = [i for i in Categories if "DK" in i["Name"]]
    URLS_Ex = ["https://www.nordpoolgroup.com"+i["Url"] for i in DKK]

    # We obain two datasets for 2015 and by inspecting the data sets we realize that the set called: 
    # 'https://www.nordpoolgroup.com/48e277/globalassets/marketdata-excel-files/exchange-dk-connections_2015_hourly2.xls',
    # is the data of intereset
    
    URLS_Ex.pop(2)
    df_Exchange = [pd.read_html(i)[0].droplevel(level = 0, axis = 1).droplevel(level = 0, axis = 1) for i in URLS_Ex]

    # Concatenate the dataframes, rename, fix misplaced decimal pointer, and format time stamp
    df_Ex = pd.concat(df_Exchange)
    df_Ex = df_Ex.rename(columns = {"Unnamed: 0_level_2": "date"})
    df_Ex["date"] = pd.to_datetime(df_Ex["date"], format = "%d-%m-%Y")

    df_Ex["hour"] = df_Ex["Hours"].str[5:]
    df_Ex.drop('Hours', axis = 1, inplace = True)
    df_Ex["hour"] = pd.to_datetime(df_Ex["hour"], format = '%H').dt.hour

    cols = ["DK - DE", "DK - NO", "DK - SE", "DK1 - SE3", "DK2 - SE4", "DK1 - DE", "DK2 - DE", "DK1 - DK2", "DK1 - NL"]
    df_Ex[cols] = df_Ex[cols]/100

    # Set index equal to time stamp 
    df_Ex.index = pd.to_datetime(df_Ex["date"].apply(str)+' '+pd.to_datetime(df_Ex["hour"], format = '%H').dt.time.apply(str))

    return df_Ex

### Merging all Nordpool data together into one DataFrame ###
# Creating function to merge dataframes from the Nordpool database
def mergeNordpool(list_of_dfs):
    """ Function that merges all of the Nordpool dataframes into one DataFrame on date and hour via an outer join """
    df_Nordpool = list_of_dfs[0]
    for df in list_of_dfs[1:]:
        df_Nordpool = pd.merge(df_Nordpool, df, how = "outer", on = ["date", "hour"])
    df_Nordpool.reset_index(drop = True)
    return df_Nordpool


# Weather data from DMI API

In [ ]:
# Constants
metObsAPIKey = 'c4503ba1-28d4-45c5-850a-974e98bbb3e0'
climateDataAPIKey = 'ac27b332-bde2-4138-a53e-f0ca82cf3667'

## Defining functions to be used for gathering and handling the DMI weather data

In [ ]:
### Defining a function to extract data from the DMI metObs APi ### 
def getMetObsData(stat, start_date, end_date, stationId = '06183', apiKey = metObsAPIKey):
    """ Function that takes a parameter ID and searches for all observations from DMI metObs APi 
    for the given dates, where the date format is given as 'YYYY-MM-DD', and for the given station ID """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId={stationId}&parameterId={id}&bbox=7,54,16,58&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0

    # Loops until it has observations for the start date. This is done due to the max response size being 300.000,
    # and if one goes far enough back for observations with frequencies of 10 min., then we pass the response size,
    # so we need to make another response and append this to the resulting DataFrame
    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            res, _ = conn.get(temp_url, 'DMI Data')
            json_dmi = res.json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            # Drops any accidental duplicates before converting to CET localization
            temp_df['observed'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['observed'])
            temp_df = temp_df.tz_convert('CET')

            # Checks if the above stated condition is fulfilled
            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df_met_obs_func = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df_met_obs_func = pd.concat([df_met_obs_func,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                continue
            else:
                if local_counter == 1:
                    df_met_obs_func = temp_df
                elif local_counter > 1:
                    df_met_obs_func = pd.concat([df_met_obs_func,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
        
        # Checks if all observations were retrieved in the first response, and if so, return temp_df, otherwise df_met_obs_func
        # would be defined, so return this DF then.
        except KeyError as e:

            try:
                df_met_obs_func
            except NameError:
                return temp_df
            else:
                return df_met_obs_func
        
    return df_met_obs_func

### Defining a function that will fill in the NaN with data from other DMI weather stations for the metObs data###        
def metObsNaFiller(stat, start_date, end_date):
    """This function takes a list of parameters, a start date and a end date and then 
    fills in the NaN from metObs data with data from other DMI weather stations"""
    stationList = ['06184', '06186', '06187', '06188']

    # Create the DF for the first and default station
    df = getMetObsData(stat, start_date, end_date)

    # Fill in the NAs with observations from all stations listed above
    for station in stationList:
        df = df.combine_first(getMetObsData(stat, start_date, end_date, stationId = station))

    return df

### Defining a function to extract data from the DMI ClimateData APi ### 
def getClimateData(stat, start_date, end_date, stationId = '06184',apiKey = climateDataAPIKey):
    """ Function that takes a parameter ID and searches for all observations from DMI Climate API
    for the given dates, where the date format is given as 'YYYY-MM-DD', and for the given station ID """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/climateData/collections/stationValue/items?timeResolution=hour&limit=300000&stationId={stationId}&parameterId={id}&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0

    # Loops until it has observations for the start date. This is done due to the max response size being 300.000,
    # and if one goes far enough back for observations with frequencies of 10 min., then we pass the response size,
    # so we need to make another response and append this to the resulting DataFrame
    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            res, _ = conn.get(temp_url, 'DMI Data')
            json_dmi = res.json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)

            # Drops any accidental duplicates before converting to CET localization, which is done in the transformation function,
            # as otherwise the below if condition would not be satisfied due to way the observations are returned in the response object.
            temp_df['to'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['to'])

            # Checks if the above stated condition is fulfilled
            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df_climate_func = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df_climate_func = pd.concat([df_climate_func,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                continue
            else:
                if local_counter == 1:
                    df_climate_func = temp_df
                elif local_counter > 1:
                    df_climate_func = pd.concat([df_climate_func,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break

        # Checks if all observations were retrieved in the first response, and if so, return temp_df, otherwise df_met_obs_func
        # would be defined, so return this DF then.
        except KeyError as e:

            try:
                df_climate_func
            except NameError:
                return temp_df
            else:
                return df_climate_func
        
    return df_climate_func

### Defining a function that will fill in the NaN with data from other DMI weather stations for the Climate data ###        
def ClimateNaFiller(stat, start_date, end_date):
    """This function takes a list of parameters, a start date and a end date and then 
    fills in the NaN from Climate data with data from other DMI weather stations"""
    stationList = ['06181', '06186', '06187', '06188']

    # Create the DF for the first and default station
    df = getClimateData(stat, start_date, end_date)

    # Fill in the NAs with observations from all stations listed above
    for station in stationList:
        df = df.combine_first(getClimateData(stat, start_date, end_date, stationId = station))

    return df

### Transforming the metObs data so that it goes from long format to wide format and has date and hour columns ###
def transformMetObsData(df):
    """This function transforms the metObs data by adding date and hour columns, dropping unnecessary columns,
    and makes the format from long format to wide format"""
    df = df
    df['hour'] = df.index.hour
    df['date'] = df.index.date
    df = df.drop(['created', 'stationId'], axis = 1)\
            .sort_values(by = ['date', 'hour'])\
            .copy()
            #.drop_duplicates(['parameterId', 'date','hour'])

    df_new = df.groupby(['parameterId', 'date', 'hour'])['value'].mean()\
            .unstack(level = 0)\
            .reset_index().rename(columns={df.index.name:None})

    return df_new
    
### Transforming the Climate data so that it goes from long format to wide format and has date and hour columns ###
def transformClimateData(df):
    """This function transforms the Climate data by adding date and hour columns, dropping unnecessary columns,
    and makes the format from long format to wide format"""
    df = df
    df = df.tz_convert('CET')
    df['hour'] = df.index.hour
    df['date'] = df.index.date
    df = df.drop(['calculatedAt', 'created', 'from', 'qcStatus', 'timeResolution', 'validity'], axis = 1)\
            .sort_values(by = ['date', 'hour'])\
            .copy()
    
    df_new = df.groupby(['parameterId', 'date', 'hour'])['value'].mean()\
            .unstack(level = 0)\
            .reset_index().rename(columns={df.index.name:None})

    return df_new

### Function that handles the total procesing of metObs data ###
def metObsPipeline(stats, start_date, end_date):
    """This function handles all of the metObs data gathering and transformation given a list of parameters,
    a start date and a end date. Also provides a progress bar to the data gathering process"""
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = metObsNaFiller(stat, start_date, end_date)

    new_df_met = pd.concat(dict_df.values())
    transform_df_met = transformMetObsData(new_df_met)
    return transform_df_met

### Function that handles the total procesing of Climate data ###
def climatePipeline(stats, start_date, end_date):
    """This function handles all of the Climate data gathering and transformation given a list of parameters,
    a start date and a end date. Also provides a progress bar to the data gathering process"""
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = ClimateNaFiller(stat, start_date, end_date)

    new_df_climate = pd.concat(dict_df.values())
    transform_df_climate = transformClimateData(new_df_climate)
    return transform_df_climate

### Function that merges the metObs and Climate data together into a single DataFrame ###
def merger(df_met, df_climate, start_date, end_date):
    """This function merges together all of the DMI weather data from the Climate and metObs APIs,
    and makes sure that there's an index for every hour from start till end date."""
    index1 = pd.date_range(start=start_date, end=end_date, freq = 'H').to_pydatetime().tolist()
    dfindex = pd.DataFrame(index = index1)
    dfindex['date'] = dfindex.index.date
    dfindex['hour'] = dfindex.index.hour
    merge_df = dfindex.reset_index()\
            .merge(df_met, how="left", on = ['date', 'hour'])\
            .merge(df_climate, how = 'left', on = ['date', 'hour'])\
            .set_index('index')\
            .sort_index()

    return merge_df

### This function handles all of the DMI data handling by calling the above functions ###
def total_DMI_pipeline(met_stats, climate_stats, start_date, end_date):
    """This function merely handles the total data gathering and wrangling process for the DMI data and adds info messages along the way to track progress"""
    print('INFO: Running metObs pipeline')
    df_met = metObsPipeline(met_stats, start_date, end_date)
    print('INFO: Running Climate data pipeline')
    df_climate = climatePipeline(climate_stats, start_date, end_date)
    print('INFO: Merging the climate and metObs data together')
    df_total = merger(df_met, df_climate, start_date, end_date)
    print('INFO: Done')
    return df_total

# DEPRECIATED DMI FUNCTIONS
The following functions are merely kept to show the missing data that was encountered during the old DMI data gathering approach described in the paper.

In [ ]:
def oldClimateData(stat, start_date, end_date, stationId = '06184',apiKey = climateDataAPIKey):
    """
    Function that takes a parameter ID and searches for all observations from DMI
    for the given dates, where the date format is given as 'YYYY-MM-DD'
    """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/climateData/collections/stationValue/items?timeResolution=hour&limit=300000&stationId={stationId}&parameterId={id}&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0
    stationCounter = 0
    stationList = ['06181', '06186', '06187', '06188']

    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            res, _ = conn.get(temp_url, 'DMI Data')
            json_dmi = res.json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            temp_df['to'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['to'])
            #temp_df = temp_df.tz_convert('CET')

            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                #warnings.warn('Loop loop date wrong')
                continue
            else:
                if local_counter == 1:
                    df = temp_df
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
            


        except KeyError as e:

            try:
                stationId = stationList[stationCounter]
                stationCounter += 1
                #warnings.warn('Loop loop station wrong')
                continue
            except IndexError as i:
                if ErrorCounter == 1:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations")
                else:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
                return None
        
    return df


In [ ]:
def oldMetObsData(stat, start_date, end_date, stationId = '06183', apiKey = metObsAPIKey):
    """
    Function that takes a parameter ID and searches for all observations from DMI
    for the given dates, where the date format is given as 'YYYY-MM-DD'
    """

    # Define constants to be used in the function
    url= lambda API_KEY, id, startDate, endDate, stationId: f'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?limit=300000&stationId={stationId}&parameterId={id}&bbox=7,54,16,58&datetime={startDate}T00:00:00Z/{endDate}T00:00:00Z&api-key={API_KEY}'
    local_counter = 1
    ErrorCounter = 0
    stationCounter = 0
    stationList = ['06184', '06186', '06187', '06188']

    while True:
        try:
            temp_url = url(apiKey, stat, start_date, end_date, stationId)
            ErrorCounter += 1

            res, _ = conn.get(temp_url, 'DMI Data')
            json_dmi = res.json()
            prop = [item['properties'] for item in json_dmi['features']]
            temp_df = pd.DataFrame(prop)
            
            temp_df['observed'].drop_duplicates(inplace = True)
            temp_df.index = pd.to_datetime(temp_df['observed'])
            temp_df = temp_df.tz_convert('CET')

            if temp_df.index[-1].date() != datetime.date.fromisoformat(start_date):
                            
                if local_counter == 1:
                    df = temp_df.copy()
                    local_counter += 1
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in IF")

                end_date = temp_df.index[-1].date().strftime("%Y-%m-%d")
                continue
            else:
                if local_counter == 1:
                    df = temp_df
                elif local_counter > 1:
                    df = pd.concat([df,temp_df])
                else:
                    warnings.warn("Something went wrong with the local counter in ELSE")

                break
            


        except KeyError as e:

            try:
                stationId = stationList[stationCounter]
                stationCounter += 1
                continue
            except IndexError as i:
                if ErrorCounter == 1:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations")
                else:
                    warnings.warn(f"The requested stat {stat} isn't available for any default stations for the daterange: {start_date} - {end_date}")
                return None
        
    return df

        
    

In [ ]:

### Function that handles the total procesing of metObs data ###
def oldMetObsPipeline(stats, start_date, end_date):
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = oldMetObsData(stat, start_date, end_date)

    new_df_met = pd.concat(dict_df.values())
    transform_df_met = transformMetObsData(new_df_met)
    return transform_df_met

### Function that handles the total procesing of Climate data ###
def oldClimatePipeline(stats, start_date, end_date):
    dict_df = dict()
    for stat in tqdm(stats):
        dict_df[stat] = oldClimateData(stat, start_date, end_date)

    new_df_climate = pd.concat(dict_df.values())
    transform_df_climate = transformClimateData(new_df_climate)
    return transform_df_climate

def old_total_DMI_pipeline(met_stats, climate_stats, start_date, end_date):
    print('INFO: Running metObs pipeline')
    df_met = oldMetObsPipeline(met_stats, start_date, end_date)
    print('INFO: Running Climate data pipeline')
    df_climate = oldClimatePipeline(climate_stats, start_date, end_date)
    print('INFO: Merging the climate and metObs data together')
    df_total = merger(df_met, df_climate, start_date, end_date)
    print('INFO: Done')
    return df_total

# Getting commodity data from https://www.investing.com/
In the below code, you will need to change the ***path*** variable to fit the location of your ChromeDriver, otherwise the code will fail.

In [ ]:
### List of all the commodities we will exctract data on from the website ###
list_of_commodities = ['gold', 'silver', 'crude-oil', 'rotterdam-coal-futures', 'natural-gas', 'carbon-emissions']

### Function that scrapes data for a given commodity and date range from the website using Selenium ###
def getCommodityData(commodity, startDate, endDate):
    """Function that scrapes data for a given commodity and date range from investing.com using Selenium"""

    # The commodity has to be specified in the form of '-'.join(commodity_name) e.g. rotterdam-coal-futures or crude-oil.
    url = lambda id: f'https://www.investing.com/commodities/{id}-historical-data'
    temp_url = url(commodity)
    startDate = datetime.datetime.strptime(startDate, "%Y-%m-%d").strftime('%m/%d/%Y')
    endDate = datetime.datetime.strptime(endDate, "%Y-%m-%d").strftime('%m/%d/%Y')

    # Insert the path to your Chrome Driver.
    path = '/Users/simonjuulhansen/Desktop/Polit/ISDS/chromedriver'
    driver = webdriver.Chrome(executable_path=path)

    # Create a request interceptor that will contain the request header for Selenium requests.
    # To confirm that the header is present, in the driver window go to https://httpbin.org/headers
    def interceptor(request):
        request.headers['Contact_details'] = 'qxd466@alumni.ku.dk'
        request.headers['Purpose'] = "Dear recipient, we've scraped your data with the purpose of creating an academic report on electricity prices and if it's possible to predict these. Please contact us at via the contact details, if there's any issue with this. Best regards."

    # Set the interceptor on the driver
    driver.request_interceptor = interceptor
    
    driver.get(temp_url)
    
    time.sleep(3)

    # Accept cookies
    cookies = driver.find_element_by_css_selector('#onetrust-accept-btn-handler')
    ActionChains(driver).click(cookies).perform()

    time.sleep(2)

    # Opens the date range menu
    menu = driver.find_element_by_css_selector('#widgetFieldDateRange')
    ActionChains(driver).click(menu).perform()

    # Enters the date values. The dates have to be in the form of DD/MM/YYYY e.g. 01/01/2000.
    # This formatting is done in the above conversion of the start and end date from YYYY-MM-DD.
    inputElement1 = driver.find_element_by_id("startDate")
    inputElement1.clear()
    inputElement1.send_keys(startDate)

    inputElement2 = driver.find_element_by_id("endDate")
    inputElement2.clear()
    inputElement2.send_keys(endDate)

    # Applies the new dates
    driver.find_element_by_id("applyBtn").click();

    time.sleep(3)

    # Gets the results from the webpage
    results = driver.find_elements_by_xpath('//*[(@id = "curr_table")]//td')

    # Store the results in a DataFrame
    d = defaultdict(list)
    date_counter = 0
    value_counter = 1
    while True:
        try:
            d['date'].append(results[date_counter].text)
            d[commodity].append(results[value_counter].text)
            date_counter += 7
            value_counter += 7
        except IndexError:
            break
    df = pd.DataFrame(d)
    df['date'] = pd.to_datetime(df['date'], format = '%b %d, %Y')

    driver.quit()
    
    return df

### Function that gathers all of the commodity data and puts them into one single DataFrame, where NaN has been forward filled (daily data) ###
def getAllCommodities(commodities, startDate, endDate):
    """Function that gathers all of the commodity data and puts them into one single DataFrame, where NaN has been forward filled (daily data)"""

    dict_df = dict()

    for i in tqdm(commodities):
        print(f'INFO: Gathering commodity data for {i}')
        dict_df[i] = getCommodityData(i, startDate, endDate)

    date_range = pd.date_range(start=startDate, end=endDate, freq = 'D').to_pydatetime().tolist()
    df = pd.DataFrame(date_range, columns = ['date'])

    for i in dict_df.values():
        df = df.merge(i, on = 'date', how = 'left')
        df[i.columns[-1]] = df[i.columns[-1]].str.replace(',','').astype('float')

    df = df.ffill()
    return df

# Total data gathering and handling

### Running the below cell will gather and wrangle all of the data from DMI, Nordpool and investing.com.
This might take a while, e.g. 20 min.
Remember that the commodity price functions will fail unless you have amended the path in the above cell to match the path to your ChromeDriver.

Furthermore, sometimes the scraping-class Connector fails while collection DMI data for cloud_cover due to a timeout (HTTPSConnectionPool(host='dmigw.govcloud.dk', port=443): Read timed out. (read timeout=30)). This mainly happens when DMI has cloud server problems. Refer to https://confluence.govcloud.dk/display/FDAPI/Operational+Status+of+API to see the server status, as this is most likely the reason if the DMI functions are failing

***Selenium***: do not move your cursor between the Selenium driver start-up and the selection of dates, as this will prompt a log-in pop-up that Selenium will not close and thus cause the program to fail. In general, just leave your PC be while Selenium runs for optimal perfomance and no accidental failure.

In [ ]:
startDate = '2016-04-01'
endDate = '2021-08-01'

# Gathering Nordpool data 
#df_Nordpool = mergeNordpool([getElspotPrices(), ConsumptionPrognosis(), ProductionPrognosis(), WindPrognosis(), ExchangeConnections()])

# Gathering DMI weather data
metObs_listV2 = [ 'wind_max_per10min_past1h', 'temp_soil_max_past1h', 'cloud_cover']
climate_listV1 = ['bright_sunshine', 'mean_radiation', 'mean_pressure', 'acc_precip', 'temp_grass', 'mean_relative_hum', 'mean_temp', 'mean_wind_speed' ,'mean_wind_dir']
#df_dmi = total_DMI_pipeline(metObs_listV2, climate_listV1, startDate, endDate)

# Gathering commodity data
list_of_commodities = ['gold', 'silver', 'crude-oil', 'rotterdam-coal-futures', 'natural-gas', 'carbon-emissions']
commodities_df = getAllCommodities(list_of_commodities, startDate, endDate)

# Snippet of Data Analysis

## DMI Data

The below gathers the ***old*** DMI data, i.e., where we were gathering data and returning only a DataFrame for a single station and not a combination of data from all given stations. This, as can be seen below, does give us plenty of NA's as the DMI data is far from *pretty*.

In [ ]:
df_old_dmi = old_total_DMI_pipeline(metObs_listV2, climate_listV1, startDate, endDate)

In [ ]:
list_of_ylabels_dmi = ['%', '°C', 'm/s', 'mm', 'minutes/hours', 'hPa', 'W/m^2', '%', '°C', 'degrees', 'm/s', '°C']
axs = df_old_dmi[df_old_dmi.columns[2:]].plot(subplots=True, layout=(4,3), figsize = (20,10), sharex=True, xlabel = '');


for ax in range(len(axs.flat)):
    axs.flat[ax].set( ylabel=list_of_ylabels_dmi[ax])

Below is the new approach to gathering DMI data, i.e., filling in all NA's with observations from other stations to combine one DataFrame

In [ ]:
list_of_ylabels_dmi = ['%', '°C', 'm/s', 'mm', 'minutes/hours', 'hPa', 'W/m^2', '%', '°C', 'degrees', 'm/s', '°C']

axs = df_dmi[df_dmi.columns[2:]].plot(subplots=True, layout = (4,3), figsize = (20,10), sharex=True, xlabel = '');
for ax in range(len(axs.flat)):
    axs.flat[ax].set( ylabel=list_of_ylabels_dmi[ax])

In [ ]:
for i in df_dmi.columns:
    num_na = len(df_dmi[i]) - len(df_dmi[i].dropna())
    print(f"{i}: {num_na}")

The above shows no signs of outliers or unordinary data that causes any concerns. All of the data conforms to the data descriptions on DMI's [API website]("https://confluence.govcloud.dk/display/FDAPI/Danish+Meteorological+Institute+-+Open+Data"). Both the cloud_cover spikes and mean_relative_hum downward spikes appear to merely be extreme observations that are not the result of any measurement errors.

If one looks at the missing data, one will see that the NA's do not appear more consecutively than 2 hours, therefore we've decided to forward fill all of the NA's in the below merging function.

## Nordpool data

In [ ]:
# Makes the index to datetime to give an easy datetime x-axis ticks for the below graphs
df_Nordpool.index = pd.to_datetime(df_Nordpool["date"].apply(str)+' '+pd.to_datetime(df_Nordpool["hour"], format = '%H').dt.time.apply(str))

In [ ]:
# Restoring default plt settings
plt.rcParams.update(plt.rcParamsDefault)
plt.rc('legend',fontsize=11) # using a size in points

# Only plot for the dates that we've collected commodity prices and weather data for
df_Nordpool[df_Nordpool.index.isin(pd.date_range(start = startDate, end = endDate, freq = 'H'))]\
            .drop(['date', 'hour', 'DK2'], axis = 1)\
            .plot(subplots=True, layout = (9,2), figsize = (10,20), sharex=True );

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}

plt.rc('font', **font)

df_Nordpool[df_Nordpool.index.isin(pd.date_range(start = startDate, end = endDate, freq = 'H'))]['DK2'].plot(figsize = (20,10), ylabel = 'Price in DKK per MWh')

In [ ]:
for i in df_Nordpool.columns:
    num_na = len(df_Nordpool[df_Nordpool.index.isin(pd.date_range(start = startDate, end = endDate, freq = 'H'))][i]) - len(df_Nordpool[df_Nordpool.index.isin(pd.date_range(start = startDate, end = endDate, freq = 'H'))][i].dropna())
    print(f"{i}: {num_na}")

Based on the above figure and no. of NA's, we've decided to remove DK1-NL from our dataset. This is done in the below function that merges all 3 datasources together. 

If one looks at the missing data, one will see that the NA's do not appear more consecutively than 2 hours, therefore we've decided to forward fill all of the NA's in the below merging function.

Other than that we think that the transformation of the dataset is good as is, so we'll leave it until merging.


## Commodity price data

In [ ]:
commodities_df.set_index('date').plot(subplots=True, layout = (2,3), figsize = (20,10), sharex=True, xlabel = '', ylabel='Closing price in US Dollars');

In [ ]:
for i in commodities_df.columns:
    num_na = len(commodities_df[i]) - len(commodities_df[i].dropna())
    print(f"{i}: {num_na}")

The above graph shows no signs of mysterious nor unusual data. The commodity data is still only daily observations until the merging happens below. The crude oil drop in start Q2 2020 is of course not a fault of wrong data, however, it was the actual crude oil future price at the time. (https://www.cnbc.com/2020/06/16/how-negative-oil-prices-revealed-the-dangers-of-futures-trading.html)

## The below transforms all of the above dataframes into a single master DataFrame

In [ ]:
def total_dataframe(df_nordpool, df_dmi, df_commodities):

    list_of_commodities = ['gold', 'silver', 'crude-oil', 'rotterdam-coal-futures', 'natural-gas', 'carbon-emissions']

    # Below we're merging all of the data together into one DataFrame from all 3 input DF's
    df_nordpool_new = df_nordpool.copy()
    df_nordpool_new['date'] = df_nordpool_new['date'].astype('string')

    df_dmi_new = df_dmi.copy()
    df_dmi_new['date'] = df_dmi_new['date'].astype('string')

    df_commodities_new = df_commodities.copy()
    df_commodities_new['date'] = df_commodities_new['date'].astype('string')

    first_merge = df_dmi_new.merge(df_nordpool_new, on = ['date', 'hour'], how = 'left')
    second_merge = first_merge.merge(df_commodities_new, on = 'date', how = 'left')

    ## Deleting unneeded columns, NaN's, duplicates and dealing with daylight savings.
    # The duplicates were only for changes to wintertime, where there were one duplicate for each year at 3 AM, but with different values
    # for each of the 3 AM observations due to winter time.
    # The mean of those two were taken and are replacing the duplicated hours
    second_merge.drop(['DK1 - NL', 'DK1 - DE', 'DK1 - SE3', 'DK2 - SE4', 'DK2 - DE'], axis = 1, inplace = True)
    new_mean = second_merge[second_merge.duplicated(['date', 'hour'])].groupby(['date', 'hour']).mean().reset_index()
    second_merge.drop_duplicates(['date', 'hour'],  keep = False, inplace = True)
    df_concat = pd.concat([second_merge, new_mean]).sort_values(['date', 'hour']).reset_index(drop = True)

    # We've also replaced the summer time NA's with the mean of the 2nd and 4th hour to create a value for the 3rd hour of the day
    # for all variables except for the commodities, as these does not suffer from this issue, as they're daily observations.
    # They are merely forward filled below.
    list1 = df_concat.columns.tolist()
    list2 = [item for item in list1 if item not in list_of_commodities+['date','hour']]
    for i in list2:
        df_concat[i] = (df_concat[i].ffill() + df_concat[i].bfill())/2

    # Forward fill the commodity data to make the price of all hours of the day equal to the daily price
    df_concat = df_concat.ffill()

    # Below we're generating hour, weekday and month dummies and appending them to the DataFrame 
    # while also converting the index to a Datetime object in the main DataFrame
    df_concat.index = pd.to_datetime(df_concat['date'].apply(str)+' '+pd.to_datetime(df_concat['hour'], format = '%H').dt.time.apply(str))
    df_concat['month'] = df_concat.index.month
    df_concat['weekday'] = df_concat.index.weekday

    d = defaultdict(list)

    d[0] = pd.get_dummies(df_concat['hour'], prefix = 'hour')
    d[1] = pd.get_dummies(df_concat['weekday'], prefix = 'weekday')
    d[2] = pd.get_dummies(df_concat['month'], prefix = 'month')
    df_join = df_concat.copy()
    for i in d.values():
        df_join = df_join.join(i)


    # Below we're lagging the variables to comply with the time series element of our regression
    # as to not create future leakage

    # Lagging the DK2 prices 48 times and only keeping the 24-48 lags
    lagged_DK2 = lagmat(df_join['DK2'], 48, use_pandas = True, trim = 'both')
    lagged_DK2 = lagged_DK2[lagged_DK2.columns[23:]]

    # Lagging the commodity data once by 24 hours
    d_lag_commodity = {}
    for i in list_of_commodities:
        d_lag_commodity[i] = lagmat(df_join[i], 24, use_pandas = True, trim = 'both')
        d_lag_commodity[i] = d_lag_commodity[i][d_lag_commodity[i].columns[23]]
    df_lag_commodity = pd.concat(d_lag_commodity.values(), axis = 1)

    # Lagging the exchange data once
    exchange_list = ['DK net exchange', 'DK1 net exchange', 'DK2 net exchange', 'DK - DE',
       'DK - NO', 'DK - SE', 'DK1 - DK2']
    d_lag_exchange = {}
    for i in exchange_list:
        d_lag_exchange[i] = lagmat(df_join[i], 24, use_pandas = True, trim = 'both')
        d_lag_exchange[i] = d_lag_exchange[i][d_lag_exchange[i].columns[23]]
    df_lag_exchange = pd.concat(d_lag_exchange.values(), axis = 1)

    # Appending all lags to the DataFrame
    list_of_df_lags = [df_lag_commodity, df_lag_exchange, lagged_DK2]
    for i in list_of_df_lags:
        df_join = df_join.join(i)
    
    # Dropping original non-lagged columns except the target, DK2
    df_join = df_join.dropna().drop(exchange_list+list_of_commodities+['month', 'weekday', 'date', 'hour'], axis = 1)
    df_join.drop(['weekday_0', 'hour_0', 'month_1'], axis = 1, inplace = True)

    # Adding a trend column to detrend
    df_join['trend'] = [i+1 for i in range(len(df_join))]

    return df_join

In [ ]:
df_all_data = total_dataframe(df_Nordpool, df_dmi, commodities_df)

In [ ]:
df_all_data

# Logging 
The below is a short overview of the logging activity from the above scraping and API request exercises

In [ ]:
df_log = pd.read_csv('log.csv', sep = ';')

In [ ]:
df_log.index = pd.to_datetime(df_log.t, unit='s')\
               .dt.strftime('%H-%M-%S')

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
f,ax = plt.subplots(1,2, figsize=(13,4))

ax[0].plot('response_code', data = df_log[df_log['project'] == 'DMI Data'])
ax[1].plot( 'response_code', data = df_log[df_log['project'] == 'Nordpool Data']) 

ax[0].xaxis.set_major_locator(plt.MaxNLocator(5))

ax[0].set_xlabel('time') # Choose title x-axis
ax[1].set_xlabel('time') # Choose title x-axis

ax[0].set_ylabel('Response code') # Choose title y-axis
ax[1].set_ylabel('Response code') # Choose title y-axis

ax[0].set_title('DMI Data')
ax[1].set_title('Nordpool Data')
    

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
f,ax = plt.subplots(1,2, figsize=(13,4))

ax[0].plot('delta_t', data = df_log[df_log['project'] == 'DMI Data'])
ax[1].plot( 'delta_t', data = df_log[df_log['project'] == 'Nordpool Data']) 

ax[0].xaxis.set_major_locator(plt.MaxNLocator(5))

ax[0].set_xlabel('time') # Choose title x-axis
ax[1].set_xlabel('time') # Choose title x-axis

ax[0].set_ylabel('Response time') # Choose title y-axis
ax[1].set_ylabel('Response time') # Choose title y-axis

ax[0].set_title('DMI Data')
ax[1].set_title('Nordpool Data')

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
f,ax = plt.subplots(1,2, figsize=(13,4))

ax[0].plot('response_size', data = df_log[df_log['project'] == 'DMI Data'])
ax[1].plot( 'response_size', data = df_log[df_log['project'] == 'Nordpool Data']) 

ax[0].xaxis.set_major_locator(plt.MaxNLocator(5))
ax[1].xaxis.set_major_locator(plt.MaxNLocator(5))

ax[0].set_xlabel('time') # Choose title x-axis
ax[1].set_xlabel('time') # Choose title x-axis

ax[0].set_ylabel('Response size') # Choose title y-axis
ax[1].set_ylabel('Response size') # Choose title y-axis

ax[0].set_title('DMI Data')
ax[1].set_title('Nordpool Data')

# Machine Learning

## LASSO regression using grid search CV 
The following code is highly inspired by "https://github.com/fnauman/timeseries/blob/master/candydata/xgboost_pipeline_candy.ipynb" but modified to fit our data. The times series handling and use of TimeSeriesSplit is confirmed by https://github.com/carl24k/fight-churn


In [ ]:
# Defining different measures to evaluate performance 
# MAPE
def mean_absolute_percentage_error(y_true, y_pred, eps = 1e-8): 
    return np.mean(np.abs((y_true - y_pred) / (y_true))) * 100 

mape = make_scorer(mean_absolute_percentage_error, 
                   greater_is_better=False)

# SMAPE
def symmetric_mape(y_true, y_pred, eps = 1e-8):
    summ = ((np.abs(y_true) + np.abs(y_pred)) + eps)
    return np.mean(np.abs(y_pred - y_true) / summ) * 100

smape = make_scorer(symmetric_mape, 
                    greater_is_better=False)

def print_scores(y_test, y_pred):
    print(f"R2 score: {r2_score(y_test, y_pred)}")
    print(f"MSE score: {MSE(y_test, y_pred)}")
    print(f"MAE score: {MAE(y_test, y_pred)}")
    print(f"Median AE score: {MAE(y_test, y_pred)}")
    print(f"MAPE score: {mean_absolute_percentage_error(y_test, y_pred)}")
    print(f"SMAPE score: {symmetric_mape(y_test, y_pred)}")

In [ ]:
# Split the data into a train and test set with a 80/20 split
df_train, df_test = df_all_data.iloc[:-9341], df_all_data.iloc[-9341:]

In [ ]:
# Create plot visualizing the train and test data 

# Create an axis
fig, ax = plt.subplots(figsize = (20,5))

# Plot the train and test setsa dn show
df_train["DK2"].plot(ax=ax)
df_test["DK2"].plot(ax=ax)
#plt.axvline(x=4000, color='k', linestyle='--')
ax.legend(["Train", "Test"]);
plt.show()

In [ ]:
# Create arrays for X and y for both training and testing data set. 
y_train = df_train[["DK2"]].to_numpy()
y_test  = df_test[["DK2"]].to_numpy()

X_train = df_train.copy()
X_test  = df_test.copy()

X_train.drop('DK2', inplace=True, axis=1)
X_test.drop('DK2', inplace=True, axis=1)

X_train = X_train.to_numpy()
X_test  = X_test.to_numpy()

In [ ]:
# Define the pipeline for the Lasso regression
pipe_lasso = Pipeline([
                       #('polynomial', PolynomialFeatures(degree = 2, include_bias=True, interaction_only=True)),
                       ('scale', StandardScaler()),
                       ('lasso', Lasso())
                       ])

# Set the different values of lambdas to be searched through in the CV
params_lasso = {'lasso__alpha': np.logspace(-4,4,20)}

In [ ]:
# Define the type of cross validation used in the grid search. The TimesSeriesSplit 
# is the k-fold alternative for times series keeping the natural order in the data 
# i.e. one avoid using future observations to predict the past. 
ts_cv = TimeSeriesSplit(n_splits=20).split(X_train)

# Define the pipeline for the grid search using the LASSO pipeline and defined type of CV.
# Further, we apply the relevant scoring and parallelizes the CV
gs_lasso = GridSearchCV(pipe_lasso,
                        param_grid=params_lasso,
                        scoring='neg_mean_absolute_error',
                        cv=ts_cv,
                        n_jobs=4)

In [ ]:
# Using the defined pipeline to fit our training set
gs_lasso.fit(X_train,y_train)
# And predicting using the LASSO regression model and the test data 
y_pred_lasso = gs_lasso.predict(X_test)

In [ ]:
# Plot the actual values (train and test) and predicted valuesw
fig, ax = plt.subplots(figsize = (20,5))

df_train["DK2"].plot(ax=ax)
df_test["DK2"].plot(ax=ax)

df_test["pred"] = y_pred_lasso
df_test["pred"].plot(ax=ax)

ax.legend(["Train","Test","Prediction"]);
plt.show()

In [ ]:
# Optimal lambda value from the grid search
print(f"Optimal lambda value in Lasso using grid search: {gs_lasso.best_params_}")

# Different scoring values
print_scores(y_test, y_pred_lasso)

In [ ]:
# Defining the final pipeline using the optimal lambda detected from the grid search 
finalpipe_lasso = Pipeline([
                       #('polynomial', PolynomialFeatures(degree = 2, include_bias=True, interaction_only=True)),
                       ('scale', StandardScaler()),
                       ('lasso', Lasso(alpha = gs_lasso.best_params_["lasso__alpha"]))
                       ])

# Fitting the training data and predicting on the test data
finalpipe_lasso.fit(X_train,y_train)
finalpipe_lasso.predict(X_test)

In [ ]:
# Save coefficients to a dictionary
df_new = df_all_data.drop("DK2", axis = 1).copy()
d_lasso = {}
for i in range(len(df_new.columns)):
    d_lasso[df_new.columns[i]] = list(finalpipe_lasso.named_steps['lasso'].coef_)[i]
d_lasso

## Ridge regression using grid search CV 
The following code is highly inspired by "https://github.com/fnauman/timeseries/blob/master/candydata/xgboost_pipeline_candy.ipynb" but modified to fit our data. The times series handling and use of TimeSeriesSplit is confirmed by https://github.com/carl24k/fight-churn.


In [ ]:
# Define the pipeline for the Lasso regression
pipe_ridge = Pipeline([
                       #('polynomial', PolynomialFeatures(degree = 2, include_bias=True, interaction_only=True)),
                       ('scale', StandardScaler()),
                       ('ridge', Ridge())
                       ])

# Set the different values of lambdas to be searched through in the CV
params_ridge = {'ridge__alpha': np.logspace(-4,4,20)}

In [ ]:
# Define the type of cross validation used in the grid search. The TimesSeriesSplit 
# is the k-fold alternative for times series keeping the natural order in the data 
# i.e. one avoid using future observations to predict the past. 
ts_cv = TimeSeriesSplit(n_splits=20).split(X_train)

# Define the pipeline for the grid search using the LASSO pipeline and defined type of CV.
# Further, we apply the relevant scoring and parallelizes the CV
gs_ridge = GridSearchCV(pipe_ridge,
                        param_grid=params_ridge,
                        scoring='neg_mean_absolute_error',
                        cv=ts_cv,
                        n_jobs=4)

In [ ]:
# Using the defined pipeline to fit our training set
gs_ridge.fit(X_train,y_train)
# And predicting using the LASSO regression model and the test data 
y_pred_ridge = gs_ridge.predict(X_test)

In [ ]:
# Plot the actual values (train and test) and predicted valuesw
fig, ax = plt.subplots(figsize = (20,5))

df_train["DK2"].plot(ax=ax)
df_test["DK2"].plot(alpha = 0.5, color = "orange", ax=ax)

df_test["pred"] = y_pred_ridge
df_test["pred"].plot(alpha = 0.5, color = "green", ax=ax)
ax.set_xlabel("")
ax.set_ylabel("Price pr MWH in DKK")
ax.legend(["Train","Test","Prediction"]);
plt.show()

In [ ]:
# Optimal lambda value from the grid search
print(f"Optimal lambda value in ridge using grid search: {gs_ridge.best_params_}")

# Different scoring values
print_scores(y_test, y_pred_ridge)

In [ ]:
# Defining the final pipeline using the optimal lambda detected from the grid search 
finalpipe_ridge = Pipeline([
                       #('polynomial', PolynomialFeatures(degree = 2, include_bias=True, interaction_only=True)),
                       ('scale', StandardScaler()),
                       ('ridge', Ridge(alpha = gs_ridge.best_params_["ridge__alpha"]))
                       ])

# Fitting the training data and predicting on the test data
finalpipe_ridge.fit(X_train,y_train)
finalpipe_ridge.predict(X_test)

In [ ]:
# Save coefficients to a dictionary
df_new = df_all_data.drop("DK2", axis = 1).copy()
d_ridge = {}
for i in range(len(df_new.columns)):
    d_ridge[df_new.columns[i]] = list(finalpipe_ridge.named_steps['ridge'].coef_.T)[i]
d_ridge

## Elastic net regression using grid search CV 
The following code is highly inspired by "https://github.com/fnauman/timeseries/blob/master/candydata/xgboost_pipeline_candy.ipynb" but modified to fit our data. The times series handling and use of TimeSeriesSplit is confirmed by https://github.com/carl24k/fight-churn


In [ ]:
# Define the pipeline for the Elastic Net regression
pipe_elasticnet = Pipeline([
                       #('polynomial', PolynomialFeatures(degree = 2, include_bias=True, interaction_only=True)),
                       ('scale', StandardScaler()),
                       ('elasticnet', ElasticNet())
                       ])

# Set the different values of lambdas to be searched through in the CV
params_elasticnet = {'elasticnet__alpha':np.logspace(-4,4,20), 'elasticnet__l1_ratio':np.linspace(0,1,20)},

In [ ]:
# Define the type of cross validation used in the grid search. The TimesSeriesSplit 
# is the k-fold alternative for times series keeping the natural order in the data 
# i.e. one avoid using future observations to predict the past. 
ts_cv = TimeSeriesSplit(n_splits=20).split(X_train)

# Define the pipeline for the grid search using the LASSO pipeline and defined type of CV.
# Further, we apply the relevant scoring and parallelizes the CV
gs_elasticnet = GridSearchCV(pipe_elasticnet,
                        param_grid=params_elasticnet,
                        scoring='neg_mean_absolute_error',
                        cv=ts_cv,
                        verbose = 2,
                        n_jobs=4)

In [ ]:
# Plot the actual values (train and test) and predicted valuesw
fig, ax = plt.subplots(figsize = (20,5))

df_train["DK2"].plot(ax=ax)
df_test["DK2"].plot(ax=ax)

df_test["pred"] = y_pred_elasticnet
df_test["pred"].plot(ax=ax)
ax.set_xlabel("")
ax.set_ylabel("DKK/MWh", fontsize = 14)
ax.set_title("Elastic net", fontsize = 20)
ax.legend(["Train","Test","Prediction"],loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=False, ncol=3, fontsize = 14);

plt.show()

In [ ]:
# Optimal lambda value from the grid search
print(f"Optimal lambda value in elastic net using grid search: {gs_elasticnet.best_params_}")

# Different scoring values
print_scores(y_test, y_pred_elasticnet)

In [ ]:
# Defining the final pipeline using the optimal lambda detected from the grid search 
finalpipe_elasticnet = Pipeline([
                       #('polynomial', PolynomialFeatures(degree = 2, include_bias=True, interaction_only=True)),
                       ('scale', StandardScaler()),
                       ('elasticnet', ElasticNet(alpha = gs_elasticnet.best_params_["elasticnet__alpha"], l1_ratio = gs_elasticnet.best_params_["elasticnet__l1_ratio"]))
                       ])

# Fitting the training data and predicting on the test data
finalpipe_elasticnet.fit(X_train,y_train)
finalpipe_elasticnet.predict(X_test)

In [ ]:
# Save coefficients to a dictionary
df_new = df_all_data.drop("DK2", axis = 1).copy()
d_elastic = {}
for i in range(len(df_new.columns)):
    d_elastic[df_new.columns[i]] = list(finalpipe_elasticnet.named_steps['lasso'].coef_)[i]
d_elastic

## Naive forecasting
### Note: Univariate forecast assuming that $x_{d,h} = x_{d-1,h}$
*Where d=day and h=hour*

In [ ]:
# Taking a de-tour to satisfy the young Sktime module's hate towards datetime
df_all_data.to_csv('naive.csv')

data_naive_start = pd.read_csv('naive.csv')
data_naive_start["Unnamed: 0"] = pd.to_datetime(data_naive_start["Unnamed: 0"])
data_naive_start = data_naive_start.set_index('Unnamed: 0')


data_naive = pd.read_csv('naive.csv')['DK2']
data_naive_test = data_naive[-9341:]    # Test data consists of 20% og the entire data set
data_naive_train = data_naive[:-9341]   


forecaster = NaiveForecaster(strategy='last', sp=24)    # Seasonality trajectory at 24 since prices are folowing daily seasons
forecaster.fit(data_naive_train)
pred = forecaster.predict(list(range(1,9342)))   #list(range(1,10001))

g = pd.DataFrame([data_naive_test,pred]).T.set_index(data_naive_start[-9341:].index)

print('MAE from Naive Forecast: ', MAE(g['DK2'],g['Unnamed 0']))

In [ ]:
g['Naive forecast'] = g['Unnamed 0']
g.drop('Unnamed 0', axis=1,inplace=True) 

g.reindex_like(data_naive_start[-6000:])

# Calculate error measures:
g['MSE'] = MSE(g['DK2'],g['Naive forecast'])
g['MAE'] = MAE(g['DK2'],g['Naive forecast'])

g

In [ ]:
# Plot the actual values (train and test) and predicted valuesw
fig, ax = plt.subplots(figsize = (20,5))

data_naive_start.loc['2016-04-03 00:00:00':'2020-11-24 01:00:00']["DK2"].plot(ax=ax)   #Train data platted
g.loc[:'2021-08-01 00:00:00']["DK2"].plot(ax=ax)      # Test data plotted. Insert .loc[:'2020-06-01 23:00:00'] to make graph better

g.loc[:'2021-08-01 00:00:00']["Naive forecast"].plot(ax=ax,xlabel='')  #Forecasts plotted. Inster .loc[:'2020-06-01 23:00:00'] to make graph better

ax.legend(["Train","Test","Prediction"]);

plt.show();

## Validation curve for Ridge

In [ ]:
ts_cv = TimeSeriesSplit(n_splits=20).split(X_train)

# FIT AND EVALUATE FOR DIFFERENT LAMBDAS
train_scores, test_scores = \
    validation_curve(estimator=pipe_ridge,
                     X=X_train,
                     y=y_train,
                     param_name='ridge__alpha',
                     param_range=np.logspace(-4,10,20),
                     scoring='neg_mean_absolute_error',                 
                     cv=ts_cv,
                     n_jobs = 4)


In [ ]:
# OBTAIN MSE FOR DIFFERENT LAMBDAS AND PRINT BEST
mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':np.logspace(-4,10,20)})\
              .set_index('lambda')   
print(mse_score.Validation.nsmallest(1))

In [ ]:
mse_score.plot(logx=True, logy=True, figsize=(10,6))
plt.axvline(mse_score.Validation.nsmallest(1).index, color='r', linestyle='dashed');

## Heatmap for Feature Selection

In [ ]:
data_hm = df_all_data

l = []
for column in data_hm.columns:      # Loop to remove all the variables that have been lagged several times and all dummies using REGEX
    if len(re.findall(r'\_\d|^DK2.L', column)) > 0:
        pass
    else:
        l.append(column)

data_hm = data_hm[l]              

fig, ax = plt.subplots(figsize=(10,10))
hm_all = sns.heatmap(pd.DataFrame(data_hm.corr()), cmap = 'vlag', annot = False)    # Heatmap with ALL correlations
plt.show()

## Machine learning results after Feature Selection

In [ ]:
# List of features to be excluded in the following feature selection process and machine learning models.
not_list = ['acc_precip',
'temp_grass',
'DK1 - DK2.L.24',
'DK - DE.L.24',
'cloud_cover',
'temp_soil_max_past1h',
'mean_relative_hum',
'DK1_CP',
'DK1_PP',
'DK1_WP',
'DK1 net exchange.L.24',
'DK2 net exchange.L.24',
'mean_wind_dir']

In [ ]:
# Create arrays for X and y for both training and testing data set. 
y_train = df_train[["DK2"]].to_numpy()
y_test  = df_test[["DK2"]].to_numpy()

X_train_new = df_train[df_train.columns[~df_train.columns.isin(not_list)]].copy()
X_test_new  = df_test[df_test.columns[~df_test.columns.isin(not_list)]].drop('pred',axis=1).copy()

X_train_new.drop('DK2', inplace=True, axis=1)
X_test_new.drop('DK2', inplace=True, axis=1)

X_train_new = X_train_new.to_numpy()
X_test_new  = X_test_new.to_numpy()

### Elastic net with feature selection

In [ ]:
# Define the type of cross validation used in the grid search. The TimesSeriesSplit 
# is the k-fold alternative for times series keeping the natural order in the data 
# i.e. one avoid using future observations to predict the past. 
ts_cv = TimeSeriesSplit(n_splits=20).split(X_train_new)

# Define the pipeline for the grid search using the LASSO pipeline and defined type of CV.
# Further, we apply the relevant scoring and parallelizes the CV
gs_elasticnet = GridSearchCV(pipe_elasticnet,
                        param_grid=params_elasticnet,
                        scoring='neg_mean_absolute_error',
                        cv=ts_cv,
                        verbose = 2,
                        n_jobs=4)

# Using the defined pipeline to fit our training set
gs_elasticnet.fit(X_train_new,y_train)

# And predicting using the elastic net regression model and the test data 
y_pred_elasticnet = gs_elasticnet.predict(X_test_new)

# Optimal lambda value from the grid search
print(f"Optimal lambda value in elastic net using grid search: {gs_elasticnet.best_params_}")

# Different scoring values
print_scores(y_test, y_pred_elasticnet)

### Ridge with feature selection

In [ ]:
# Define the type of cross validation used in the grid search. The TimesSeriesSplit 
# is the k-fold alternative for times series keeping the natural order in the data 
# i.e. one avoid using future observations to predict the past. 
ts_cv = TimeSeriesSplit(n_splits=20).split(X_train_new)

# Define the pipeline for the grid search using the LASSO pipeline and defined type of CV.
# Further, we apply the relevant scoring and parallelizes the CV
gs_ridge = GridSearchCV(pipe_ridge,
                        param_grid=params_ridge,
                        scoring='neg_mean_absolute_error',
                        cv=ts_cv,
                        n_jobs=4)


# Using the defined pipeline to fit our training set
gs_ridge.fit(X_train_new,y_train)
# And predicting using the LASSO regression model and the test data 
y_pred_ridge = gs_ridge.predict(X_test_new)

# Optimal lambda value from the grid search
print(f"Optimal lambda value in ridge using grid search: {gs_ridge.best_params_}")

# Different scoring values
print_scores(y_test, y_pred_ridge)

### LASSO with feature selection

In [ ]:
# Define the type of cross validation used in the grid search. The TimesSeriesSplit 
# is the k-fold alternative for times series keeping the natural order in the data 
# i.e. one avoid using future observations to predict the past. 
ts_cv = TimeSeriesSplit(n_splits=20).split(X_train_new)

# Define the pipeline for the grid search using the LASSO pipeline and defined type of CV.
# Further, we apply the relevant scoring and parallelizes the CV
gs_lasso = GridSearchCV(pipe_lasso,
                        param_grid=params_lasso,
                        scoring='neg_mean_absolute_error',
                        cv=ts_cv,
                        n_jobs=4)

# Using the defined pipeline to fit our training set
gs_lasso.fit(X_train_new,y_train)
# And predicting using the LASSO regression model and the test data 
y_pred_lasso = gs_lasso.predict(X_test_new)

# Optimal lambda value from the grid search
print(f"Optimal lambda value in Lasso using grid search: {gs_lasso.best_params_}")

# Different scoring values
print_scores(y_test, y_pred_lasso)